# 1. Estimation of the parameter $H$
We consider a sport variance $V_t$ of a *Lifted Heston model* with $n$ factors and initial curve $g_0(t)$ give by:
$$g_0(t)=V_0+\lambda\theta\sum_{i=1}^{n}c_i\int_0^t\exp(-x_i(t-s))\,\mathrm ds$$
The weights $(c_i)_{1\leq i\leq n}$ and the mean reversions $(x_i)_{1\leq i\leq n}$ are parametrized with a constant $r$ denoted by $r_n$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.special import gamma
from scipy.integrate import quad

import warnings

warnings.filterwarnings("ignore")

In [2]:
# Parameters of the Lifted Heston model
n = 20
V0 = 0.05
lambda_ = 0.3
theta = 0.05
nu = 0.1
alpha = 0.6  # H + 0.5
H = alpha - 0.5
r20 = 2.5

## Question 1 - Path of variance process

To simulate the variance in the Lifted Heston model we can use an **implicite-explicit Euler scheme**.

**Scheme:** For a simulation $\left(W_{t_k}\right)$ of a Brownian motion on a uniform partition of the $[0,T]$ with step $\Delta t = t_k - t_{k-1}$ let's consider the following scheme:

$$\hat{V}_{t_k} = g_0(t_k) + \sum_{i=1}^{n} c_i \hat U_{t_k}^i$$

$$\hat U_{t_{k+1}}^i = \frac{1}{1+x_i \Delta t} \left( \hat U_{t_k}^i - \lambda \hat V_{t_k} \Delta t + \nu \sqrt{max\left(0, \hat V_{t_k}\right)} (W_{t+1}- W_{t_k})\right)$$

First, we are going to simulate a Brownian motion $W_t$.

In [ ]:
T = 1
delta_t = 1 / 10000

time = np.linspace(0, T, int(T / delta_t) + 1)

n = 20

V0 = 0.05
lambda_ = 0.3
theta = 0.05
nu = 0.1
alpha = 0.6  # H + 0.5
H = alpha - 0.5
r20 = 2.5

In [ ]:
w = np.cumsum(np.random.normal(0, 1, len(time)))
w[0] = 0

plt.figure(figsize=(15, 5))
plt.plot(time, w)
plt.title("Brownian motion")
plt.xlabel("Time")
plt.ylabel("Value")
plt.grid()
plt.show()

To run our first variance simulation, we need the following formulas:
$$\begin{aligned}
\hat V_{t_k}&=g_0(t_k)+\sum_{i=1}^{n}c_i\hat U^i_{t_k}\\
\hat U^i_{t_k}&=\frac1{1+x_i\Delta t}\left(\hat U^i_{t_{k-1}}+\lambda\hat V_{t_k}\Delta t+\nu\sqrt{\max(0,\hat{V}_{t_k})}(W_{t_{k+1}}-W_{t_k})\right)
\end{aligned}$$
where $W_t$ is a Brownian motion, $\Delta t$ is the time step.

First, we have $n$ even and $r_{20}>1$ so we can use the geometric partition for the weights and the mean reversions.

In [ ]:
coef = (r20 ** (1 - alpha) - 1) / (gamma(alpha) * gamma(2 - alpha))
weights = np.array(
    [coef * r20 ** ((1 - alpha) * (i - 1 - n / 2)) for i in range(1, n + 1)]
)

coef = ((1 - alpha) * r20 ** (2 - alpha) - 1) / ((2 - alpha) * r20 ** (1 - alpha) - 1)
mean_reversions = np.array([coef * r20 ** (i - 1 - n / 2) for i in range(1, n + 1)])

del coef


def g0(t: float) -> float:
    """Initial curve of the model

    Args:
        t (float | np.ndarray): time

    Raises:
        ValueError: t must be positive

    Returns:
        float: curve value
    """
    if t < 0:
        raise ValueError("t must be positive")
    return V0 + lambda_ * theta * np.sum(
        [
            weight * quad(lambda s: np.exp(-x * (t - s)), 0, t)[0]
            for weight, x in zip(weights, mean_reversions)
        ]
    )


g0 = np.vectorize(g0)

In [ ]:
initial_curve_on_time = g0(time)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(time, initial_curve_on_time)
plt.title("Initial curve")
plt.xlabel("Time")
plt.ylabel("Value")
plt.grid()
plt.show()

Now we can define $\hat{V}_{t_k}$ and $\hat{U}^i_{t_k}$.

In [ ]:
v = np.zeros(len(time))
u = np.zeros((len(time), n))

for j, t in enumerate(time):
    v[j] = initial_curve_on_time[j] + np.sum(weights * u[j, :])
    if j == len(time) - 1:
        break
    for i in range(n):
        u[j + 1, i] = (
            1
            / (1 + mean_reversions[i] * delta_t)
            * (
                u[j, i]
                - lambda_ * v[j] * delta_t
                + nu * np.sqrt(np.maximum(v[j], 0)) * (w[j + 1] - w[j])
            )
        )

In [ ]:
plt.figure(figsize=(15, 5))
plt.title("Variance process")
plt.plot(time, v)
plt.xlabel("Time")
plt.ylabel("Variance")
plt.show()